# PyMentat を用いた有限要素モデルの作成

In [7]:
# PyMentatモジュールのインポート
# from py_mentat import *
import numpy as np

In [8]:
# パラメータの設定
z_list = [0, 5000, 10000]   # 基準点(mm)
t_list = [270, 260, 250]    # 板厚(mm)
r_list = [1900, 1850, 1800] # 半径(mm)
n = [10, 10, 3] # (r, theta, z)方向の分割数

In [ ]:
# 座標値の作成

# 